In [134]:
using Plots
using LaTeXStrings
using SparseArrays
using BlockDiagonals
using BenchmarkTools

In [135]:
# Define functions for the current sheet F, F', F'' 
F(x) = tanh(x)
# dF(x) = sech(x)^2
ddF(x) = -2 * tanh(x) * sech(x)^2

ddF (generic function with 1 method)

In [136]:
# Define constants
γ₋ = 0.01309
γ = 0.01309
γ₊ = 0.01309

bbγ₋ = 0.01309
bbγ = 0.01309
bbγ₊ = 0.01309

khat = 1
k₋ = 0.5
k = 0.5
k₊ = 0.5

S = 1000
v = (S / γ)^-0.5; # literal wild guess

In [137]:
function grid_spacing(domain::Tuple, n::Int, λ::Rational) 
    # λ ∈ [-1, 1]
    a, b = domain
    # compute spacing on domain: (0, 1)
    u = [((j) / (n-1)) + (λ / (2*π)) * sin(2*π*(j) / (n-1)) for j in 0:n-1]
    # now apply a linear tranformation to go from (0, 1) -> (a, b)
    return [a + (b - a) * k for k in u]
end

grid_spacing (generic function with 1 method)

In [138]:
function createA(nodes)
    n = length(nodes)
    blocks = Vector{SparseMatrixCSC{Float64, Int}}(undef, n)

    F_values = F.(nodes)
    ddF_values = ddF.(nodes)

    for i in 1:n
        F_i = F_values[i]
        ddF_i = ddF_values[i]

        mat = spzeros(6, 6)

        mat[1,1] = -S * γ₋
        mat[1,2] = S * γ₋ * F_i
        mat[2,1] = -k₋^4 / γ₋^2 - k₋^2 * ddF_i / γ₋^2
        mat[2,2] = -k₋^2

        mat[3,3] = -S * γ
        mat[3,4] = S * γ * F_i
        mat[4,3] = -k^4 / γ^2 - k^2 * ddF_i / γ^2
        mat[4,4] = -k^2

        mat[5,5] = -S * γ₊
        mat[5,6] = S * γ₊ * F_i
        mat[6,5] = -k₊^4 / γ₊^2 - k₊^2 * ddF_i / γ₊^2
        mat[6,6] = -k₊^2

        blocks[i] = mat
    end

    A = BlockDiagonal(blocks)

    # make first and last blocks zero to satisfy the BCs
    A[1:6, 1:6] .= 0.0
    A[end-5:end, end-5:end] .= 0.0

    return A
end

createA (generic function with 1 method)

In [139]:
# Function to compute finite difference weights
function stencil(x::AbstractVector{<:Real}, x₀::Real, m::Integer, p::Integer=2)
    ℓ = 0:p
    A = @. (x' - x₀)^ℓ / factorial(ℓ)
    b = ℓ .== m
    return A \ b
end

stencil (generic function with 2 methods)

In [140]:
# generic function to create a sparse banded differentiation matrix for 6 coupled ODEs
function diffmat(x::Vector{<:Real}, m::Integer, p::Integer=2)
    n = length(x)
    N = 6 * n # Number of rows/columns in the differentiation matrix
    
    if n < 3
        throw(ArgumentError("The vector length must be at least 3 for a tridiagonal matrix"))
    end
    
    D = spzeros(N, N)
    max_weights_len = p+3 # Maximum stencil length
    
    # Helper function to insert weights into the differentiation matrix
    function insert_weights!(D, row, stencil_indices, weights, var_idx)
        for (j, idx) in enumerate(stencil_indices)
            D[row, 6*(idx-1) + var_idx] = weights[j]
        end
    end
    
    # Precompute stencil indices and apply stencil weights
    for i in 1:n
        if i == 1
            stencil_indices = 1:p+2
            stencil_x = x[stencil_indices]
            x₀ = x[3]
        elseif i == n
            stencil_indices = n-(p+1):n
            stencil_x = x[stencil_indices]
            x₀ = x[n]
        else
            stencil_indices = i-1:i+1
            stencil_x = x[stencil_indices]
            x₀ = x[i]
        end
        
        weights = stencil(stencil_x, x₀, m)
        
        for var_idx in 1:6
            insert_weights!(D, 6*(i-1) + var_idx, stencil_indices, weights, var_idx)
        end
    end
    
    return D
end

diffmat (generic function with 2 methods)

In [141]:
# Function to create a sparse banded differentiation matrix for 6 coupled ODEs
function diffmat2_full(x::Vector{<:Real}, m::Integer, p::Integer=2)
    n = length(x)
    N = 6 * n # Number of rows/columns in the differentiation matrix
    
    if n < 3
        throw(ArgumentError("The vector length must be at least 3 for a tridiagonal matrix"))
    end
    
    D = spzeros(N, N)
    max_weights_len = p+3 # Maximum stencil length
    
    # Helper function to insert weights into the differentiation matrix
    function insert_weights!(D, row, stencil_indices, weights, var_idx)
        for (j, idx) in enumerate(stencil_indices)
            D[row, 6*(idx-1) + var_idx] = weights[j]
        end
    end
    
    # Precompute stencil indices and apply stencil weights
    for i in 1:n
        if i == 1
            stencil_indices = 1:p+2
            stencil_x = x[stencil_indices]
            x₀ = x[3]
        elseif i == n
            stencil_indices = n-(p+1):n
            stencil_x = x[stencil_indices]
            x₀ = x[n]
        else
            stencil_indices = i-1:i+1
            stencil_x = x[stencil_indices]
            x₀ = x[i]
        end
        
        weights = stencil(stencil_x, x₀, m)
        
        for var_idx in 1:6
            insert_weights!(D, 6*(i-1) + var_idx, stencil_indices, weights, var_idx)
        end
        
        # Modify rows in matrix directly for additional ψ 2nd derivative terms in sys of eqns
        # take the same stencil weights generated for 2nd deriv in those spots that already exist (they don't)
        # and multiply them by k^2*F / γ^2 depending on position
        D[6*(i-1) + 2, :] .+= D[6*(i-1) + 1, :] * (k₋^2 * F(x[i]) / γ₋^2)
        D[6*(i-1) + 4, :] .+= D[6*(i-1) + 3, :] * (k^2 * F(x[i]) / γ^2)
        D[6*(i-1) + 6, :] .+= D[6*(i-1) + 5, :] * (k₊^2 * F(x[i]) / γ₊^2)
    end
    
    return D
end

diffmat2_full (generic function with 2 methods)

In [142]:
# Function to create a sparse banded differentiation matrix for 6 coupled ODEs
function diffmat(x::Vector{<:Real}, m::Integer, S::Real, v::Real, khat::Real, k::Vector{<:Real}, p::Integer=2)
    n = length(x)
    N = 6 * n # Number of rows/columns in the differentiation matrix
    
    if n < 3
        throw(ArgumentError("The vector length must be at least 3 for a tridiagonal matrix"))
    end
    
    D = spzeros(N, N)
    max_weights_len = p+3 # Maximum stencil length
    
    # Helper function to insert weights into the differentiation matrix
    function insert_weights!(D, row, stencil_indices, weights, var_idx)
        for (j, idx) in enumerate(stencil_indices)
            D[row, 6*(idx-1) + var_idx] = weights[j]
        end
    end
    
    # Precompute stencil indices and apply stencil weights
    for i in 1:n
        if i == 1
            stencil_indices = 1:p+2
            stencil_x = x[stencil_indices]
            x₀ = x[3]
        elseif i == n
            stencil_indices = n-(p+1):n
            stencil_x = x[stencil_indices]
            x₀ = x[n]
        else
            stencil_indices = i-1:i+1
            stencil_x = x[stencil_indices]
            x₀ = x[i]
        end
        
        weights = stencil(stencil_x, x₀, m)
        
        for var_idx in 1:6
            insert_weights!(D, 6*(i-1) + var_idx, stencil_indices, weights, var_idx)
        end
        
        # Add first derivative terms to specific rows
        if i == 1
            # First derivative term for ψ_m in the first row
            first_deriv_weights = stencil(stencil_x, x₀, m-1)
            insert_weights!(D, 6*(i-1) + 1, stencil_indices, -S * v * (1 - (khat / k)) * first_deriv_weights, 1)
        elseif i == n
            # First derivative term for ψ_m in the fifth row
            first_deriv_weights = stencil(stencil_x, x₀, m-1)
            insert_weights!(D, 6*(i-1) + 5, stencil_indices, -S * v * (1 - (khat / k)) * first_deriv_weights, 1)
        else
            # First derivative term for ψ_{m+1} in the third row
            first_deriv_weights_plus = stencil(stencil_x, x₀, m-1)
            insert_weights!(D, 6*(i-1) + 3, stencil_indices, -S * v * (1 + (khat / k₊)) * first_deriv_weights_plus, 3)
            
            # First derivative term for ψ_{m-1} in the third row
            first_deriv_weights_minus = stencil(stencil_x, x₀, m-1)
            insert_weights!(D, 6*(i-1) + 3, stencil_indices, -S * v * (1 - (khat / k₋)) * first_deriv_weights_minus, 1)
        end
    end
    
    return D
end

diffmat1 (generic function with 2 methods)

In [143]:
dom = (-10, 10)
n = 200
mult = 2//3

x = grid_spacing(dom, n, mult);
A = createA(x);

Dx = diffmat(x, 1)
# d2_test = diffmat(x, 2);
Dxx = diffmat2_full(x, 2);
# Dxxx = diffmat(x, 3);

In [144]:
Dxx[1:6, 1:6]

6×6 SparseMatrixCSC{Float64, Int64} with 9 stored entries:
     17.8392   ⋅         ⋅       ⋅         ⋅       ⋅ 
 -26027.7     0.0        ⋅       ⋅         ⋅       ⋅ 
       ⋅       ⋅       17.8392   ⋅         ⋅       ⋅ 
       ⋅       ⋅   -26027.7     0.0        ⋅       ⋅ 
       ⋅       ⋅         ⋅       ⋅       17.8392   ⋅ 
       ⋅       ⋅         ⋅       ⋅   -26027.7     0.0

In [145]:
Dx

1200×1200 SparseMatrixCSC{Float64, Int64} with 3612 stored entries:
⎡⠻⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⣦⎦

In [146]:
#= TO-DO:
    1. create Dx with ψ derivs only, all else = 0
    2. create Dxxx with ϕ derivs only, all else = 0
    3. combine Dx, Dxx, Dxxx, and A into 1 big matrix
    4. chop off endpoint values and perform matrix shooting--
    4(cont). make sure to use LinearSolve.jl instead of A \ b
    5. check results are in ballpark
    6. estimate missing parameters, implement Muller's method 
    7. guess γ and iterate iterate iterate!
=#